## Least Cost Pipeline Construction
This notebook imports the cost distance stack generated by `Create-Cost-Stack.ipynb` and a pipeline raster to compute a least cost "feeder" pipeline configuration. The workflow for this process is:
1. Import the stack of cost distance layers (one for each biogas source): $arr\_stack$.
1. Import the pipeline raster, setting non-pipeline cells to NaN: $arr\_pipeline$.
1. Determine which pixel among the pipeline pixels has the least cost among all farm cost distances rasters. This will serve as the location of the connection point to the existing pipeline: $C_0$.
1. Determine which farm is the source of this minimum point, done by finding which layer (in the stack of cost distance rasters) has the minimum value at that location. This represents the least cost biogas source: $F_0$.
1. Compute the least cost path connecting that farm ($F_0$) to the connection point ($C_0$): $LCP_0$.
1. Update the connection point layer ($C_0$) to include the least cost path ($LCP_0$): $Pipes_0$.
1. Remove the layer associated with $F_0$ from the stack of cost distance rasters ($arr\_CDsk$) and repeat steps 4-7:
    * Locate the minimim value among all remaining cost distance rasters to cells in the Pipes layer ($C_i$)...
    * Idenfity the source farm associated with this minimum ($F_i$)...
    * Compute the least cost path from from $F_i$ to $C_i$...
    * Update the connection point layer ($Pipes_{i+1}$)...

In [34]:
#Import packages
import numpy as np
import pandas as pd
from skimage import graph
from osgeo import gdal
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
%matplotlib inline

In [68]:
#Read the cost and cost distance stacks
arrCosts = np.load('../data/DuplinCostStack.npy')
arrStack = np.load('../data/DuplinStack.npy')
dims = arrStack.shape
print("There are {0} layers in the stack ({1} rows x {2} cols)".format(dims[0],dims[1],dims[2]))

There are 464 layers in the stack (190 rows x 223 cols)


In [62]:
#Read in the pipeline rasters
ds =  gdal.Open('../data/processed/duplin_pipe_diameter.tif')
#Get the raster projection
ds_prj = ds.GetProjection()
#Get raster attributes
llx, x_size, x_angle, lly, y_angle, y_size = ds.GetGeoTransform()
#Extract Band1 as the cost array; divide by 100 to scale as a percentage
arrPipesAll = np.array(ds.GetRasterBand(1).ReadAsArray())
#Convert to a masked array (ignoring all values < 0, which are NoData)
arrPipes = np.ma.masked_array(arrPipesAll, mask=arrPipesAll < 0)
#Print info
print("Lower left coordinate = ({0:.8f},{1:.8f})".format(llx,lly))
print("Pixel size is {0:.8f}(x), {1:.8f}(y)".format(x_size,y_size))
print("Values range from {} to {}".format(arrPipes.min(),arrPipes.max()))

Lower left coordinate = (1502530.32985527,-361423.20573852)
Pixel size is 500.00000000(x), -500.00000000(y)
Values range from 0 to 36


Find the least cost farm location occurring within the pipeline layer

In [8]:
#Locate the overall minimum value and determine which layer it occurrs in and where
allMin = np.amin(arrStack)
x,y,z = np.where(arrStack == allMin)
print("The min value is in layer {}, at coordinates ({},{})".format(x,y,z))

The min value is in layer [0], at coordinates ([114],[63])


In [91]:
#Reduce the stack of cost distance layers to minimum values
arrMin = np.amin(arrStack,axis=0)
#Mask the values so only pipeline pixels are kept
arrMin_masked = np.ma.masked_array(arrMin, mask=arrPipesAll < 0)
#Find the min value of the pipeline pixels
minValue = np.amin(arrMin_masked)
#Determine the row and column where the min occurs
rMin,cMin = np.where(arrMin_masked == minValue)
#Find which layer has the min value at this location in the stack of layers
[layerMin] = np.where(arrStack[:,91,48] == minValue)
print("the Min value occurred at (row {}; column {}) in layer {}".format(rMin[0],cMin[0],layerMin[0]))

the Min value occurred at (row 91; column 48) in layer 0


Compute the LCP from that farm to the pipeline

In [ ]:
#Find the farm location, will be min in that layer
np.where(arrStack[])

In [18]:
#Compute the LCP from that location to the nearest pipe
mcpConnect = graph.MCP_Connect(arrStack[x])

In [ ]:
mcpConnect.create_connection()

In [13]:
#Remove that layer from the stack
arrStack_0 = np.delete(arrStack,x,axis=0)
arrStack_0.shape

(463, 190, 223)

In [15]:
#Get the next min
allMin = np.amin(arrStack_0)
x,y,z = np.where(arrStack == allMin)
print("The min value ({}) is in layer {}".format(allMin,x))

The min value (2.630235719465958) is in layer [1]
